In [1]:
import primer3
import pandas as pd
import numpy as np
from itertools import product
import pysam

### Задание №1
Во вложении находится  *.fasta файл с праймерами в 5’-3’ ориентации. Праймеры содержат вырожденные позиции.
Напишите скрипт, который для каждой из вырожденных структур оценивает термодинамику набора всех возможных невырожденных структур (температура плавления олигонуклеотида, энергия Гиббса, температура плавления возможных гомодимерных структур и шпилек). Приведите для каждого из рассчитанных параметров среднее значение, медианное значение, 5-й и 95-й перцентиль. Приведите указанную статистику для каждой отдельной праймерной структуры и для полного набора структур в целом.

In [2]:
degenerated_nucl = {
    'A': ['A'], 
    'T': ['T'],
    'G': ['G'],
    'C': ['C'],
    'M': ['A', 'C'],
    'R': ['A', 'G'],
    'W': ['A', 'T'],
    'S': ['C', 'G'],
    'Y': ['C', 'T'],
    'K': ['G', 'T'],
    'V': ['A', 'C', 'G'],
    'H': ['A', 'C', 'T'],
    'D': ['A', 'T', 'G'],
    'B': ['C', 'T', 'G'],
    'N': ['A', 'T', 'G', 'C']
}

In [3]:
primer_list = []
primers_fasta = pysam.Fastafile('primers_test.fasta')
for primer in primers_fasta.references:
    seq = primers_fasta.fetch(primer)
    primer_list.append(seq)
primers_fasta.close()

In [4]:
all_primers_list = []
primers_list = []
for sequence in primer_list:
    sequences = list(map(
        lambda x: ''.join(x), # Join all tuples
        product(*map( # Get all combinations 
            lambda nucl: degenerated_nucl[nucl], # Map letter to list of possible nucliotides
            sequence 
        ))
    )) 
    all_primers_list += sequences
    primers_list += [sequence] * len(sequences)

In [5]:
df = pd.DataFrame(zip(primers_list, all_primers_list), columns=['initial_primer_sequence', 'primer_sequence'])
df.head()

,initial_primer_sequence,primer_sequence
0,TTCCTCTGTCATGCCTTGG,TTCCTCTGTCATGCCTTGG
1,STCCAHGCBARMACRAANG,CTCCAAGCCAAAACAAAAG
2,STCCAHGCBARMACRAANG,CTCCAAGCCAAAACAAATG
3,STCCAHGCBARMACRAANG,CTCCAAGCCAAAACAAAGG
4,STCCAHGCBARMACRAANG,CTCCAAGCCAAAACAAACG


In [6]:
df['melting_temperature'] = df['primer_sequence'].apply(primer3.calc_tm)
df['dg_homodimer'] = df['primer_sequence'].apply(lambda x: primer3.calc_homodimer(x).dg)
df['dg_hairpin'] = df['primer_sequence'].apply(lambda x: primer3.calc_hairpin(x).dg)
df['melting_temperature_homodimer'] = df['primer_sequence'].apply(lambda x: primer3.calc_homodimer(x).tm)
df['melting_temperature_hairpin'] = df['primer_sequence'].apply(lambda x: primer3.calc_hairpin(x).tm)
df.head()

,initial_primer_sequence,primer_sequence,melting_temperature,dg_homodimer,dg_hairpin,melting_temperature_homodimer,melting_temperature_hairpin
0,TTCCTCTGTCATGCCTTGG,TTCCTCTGTCATGCCTTGG,54.923004,-2630.863687,0.0,-25.819268,0.0
1,STCCAHGCBARMACRAANG,CTCCAAGCCAAAACAAAAG,50.727199,-1864.811229,0.0,-54.920576,0.0
2,STCCAHGCBARMACRAANG,CTCCAAGCCAAAACAAATG,50.799821,-1864.811229,0.0,-54.920576,0.0
3,STCCAHGCBARMACRAANG,CTCCAAGCCAAAACAAAGG,52.648644,-1864.811229,0.0,-54.920576,0.0
4,STCCAHGCBARMACRAANG,CTCCAAGCCAAAACAAACG,53.410545,-1982.481229,0.0,-51.842523,0.0


In [7]:
df.loc[df['dg_hairpin'] == 0, 'dg_hairpin'] = None 
df.loc[df['melting_temperature_hairpin'] == 0, 'melting_temperature_hairpin'] = None 

In [8]:
num_columns = list(df.columns[2:])
stats_df = pd.concat([ 
    df[num_columns].mean().to_frame().rename(columns={0: 'mean'}).transpose(),
    df[num_columns].median().to_frame().rename(columns={0: 'median'}).transpose(),
    df[num_columns].quantile(0.05).to_frame().rename(columns={0.05: 'percentile_5'}).transpose(),
    df[num_columns].quantile(0.95).to_frame().rename(columns={0.95: 'percentile_95'}).transpose()
])


In [9]:
stats_df.head() 

,melting_temperature,dg_homodimer,dg_hairpin,melting_temperature_homodimer,melting_temperature_hairpin
mean,56.929553,-4372.511322,-561.878357,-7.508873,43.552916
median,56.821356,-3728.496145,-371.014229,-2.220760,41.993107
percentile_5,49.620555,-8913.718602,-1766.446458,-48.138620,33.225062
percentile_95,64.309283,-1744.276145,275.669542,27.896056,57.366392


In [10]:
with pd.ExcelWriter('output_1.xlsx') as writer:
    df.to_excel(writer, sheet_name='primers', index=None)
    stats_df.to_excel(writer, sheet_name='total_stats')
    df[num_columns + ['initial_primer_sequence']].groupby('initial_primer_sequence').mean().to_excel(writer, sheet_name='means')
    df[num_columns + ['initial_primer_sequence']].groupby('initial_primer_sequence').median().to_excel(writer, sheet_name='medians')
    df[num_columns + ['initial_primer_sequence']].groupby('initial_primer_sequence').quantile(0.05).to_excel(writer, sheet_name='percentiles_5')
    df[num_columns + ['initial_primer_sequence']].groupby('initial_primer_sequence').quantile(0.95).to_excel(writer, sheet_name='percentiles_95')


### Задание №2
Во вложении находится  *.tsv файл с результатом работы алгоритма BLAST (out format 6). Нуклеиновые кислоты какого вируса(-ов) были найдены в образце?
Необходимо указать полную таксономию. Определите медианные значения параметров identity, e-value, длины выравнивания для набора прочтений, отнесённых к каждой выявленной таксономической группе вирусов, а также их долю от общего числа прочтений.

In [11]:
blast_df = pd.read_csv(
    'resp1-1201107674_S85.merged.tsv', 
    sep='\t',
    names=['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 
           'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
)
blast_df.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,2,FN564434.1,98.810,84,1,0,11,94,1753925,1754008,4.930000e-32,150.0
1,2,FN564434.1,91.463,82,5,1,11,92,2527612,2527533,2.330000e-20,111.0
2,46,MH447037.1,98.592,71,1,0,43,113,310,240,2.010000e-25,126.0
3,92,LC056188.1,97.183,71,0,2,66,135,70,1,1.490000e-22,119.0
4,136,MN881965.1,93.605,344,22,0,2,345,2292,1949,1.460000e-141,514.0


In [12]:
blast_df[['qseqid']].drop_duplicates().to_csv('tax_ids.txt', index=None, header=None)

In [13]:
! cat tax_ids.txt | taxonkit reformat -I 1 > taxons.tsv

22:26:00.790 [WARN] taxid 16499 was deleted
22:26:00.790 [WARN] taxid 16676 was deleted
22:26:00.790 [WARN] taxid 25533 was deleted
22:26:00.790 [WARN] taxid 16775 was deleted
22:26:00.790 [WARN] taxid 25565 was deleted
22:26:00.790 [WARN] taxid 25757 was deleted
22:26:00.790 [WARN] taxid 16817 was deleted
22:26:00.790 [WARN] taxid 25825 was deleted
22:26:00.791 [WARN] taxid 25990 was deleted
22:26:00.791 [WARN] taxid 26135 was deleted
22:26:00.791 [WARN] taxid 46 was merged into 39
22:26:00.791 [WARN] taxid 26251 was deleted
22:26:00.791 [WARN] taxid 26270 was deleted
22:26:00.791 [WARN] taxid 16956 was deleted
22:26:00.791 [WARN] taxid 26461 was deleted
22:26:00.791 [WARN] taxid 16986 was deleted
22:26:00.791 [WARN] taxid 92 was deleted
22:26:00.791 [WARN] taxid 26725 was deleted
22:26:00.791 [WARN] taxid 17150 was deleted
22:26:00.791 [WARN] taxid 26850 was deleted
22:26:00.791 [WARN] taxid 17741 was deleted
22:26:00.791 [WARN] taxid 26871 was deleted
22:26:00.791 [WARN] taxid 17855

In [14]:
taxons_df = pd.read_csv('taxons.tsv', sep='\t', names = ['qseqid', 'taxonomy'])
taxons_df.head()

,qseqid,taxonomy
0,2,Bacteria;;;;;;
1,46,Bacteria;Pseudomonadota;Deltaproteobacteria;My...
2,92,NaN
3,136,Bacteria;Spirochaetes;Spirochaetia;Spirochaeta...
4,158,Bacteria;Spirochaetes;Spirochaetia;Spirochaeta...


In [15]:
taxons_df = taxons_df[taxons_df['taxonomy'].str.contains('Virus') & ~taxons_df['taxonomy'].isna()] 
taxons_merged = taxons_df.merge(blast_df, on='qseqid')

In [16]:
taxons_merged.loc[:, 'virus_group'] = taxons_merged['taxonomy'].apply(lambda x: x.split(';')[4])
taxons_merged = taxons_merged[taxons_merged['virus_group'] != '']

virus_stats = taxons_merged[['virus_group', 'pident', 'length', 'evalue']].groupby('virus_group').median().reset_index()
virus_stats = taxons_merged.groupby('virus_group')[['qseqid']].count()\
    .reset_index()\
    .rename(columns={'qseqid':'qseqid_count'})\
    .merge(virus_stats, on='virus_group')
virus_stats['qseqid_fraction'] = virus_stats['qseqid_count'] / blast_df.shape[0]
virus_stats.sort_values(by='qseqid_fraction', ascending=False).head(10)

,virus_group,qseqid_count,pident,length,evalue,qseqid_fraction
20,Orthomyxoviridae,314,93.5690,69.0,1.860000e-19,0.004966
31,Sedoreoviridae,156,93.1030,69.0,3.080000e-17,0.002467
16,Iridoviridae,114,95.2095,72.0,1.110000e-21,0.001803
13,Geminiviridae,107,95.3120,69.0,5.170000e-20,0.001692
3,Baculoviridae,107,95.1610,70.0,1.440000e-20,0.001692
33,Totiviridae,105,93.8780,70.0,1.860000e-19,0.001661
14,Hantaviridae,59,94.5210,80.0,2.390000e-23,0.000933
28,Retroviridae,27,89.8990,97.0,2.710000e-22,0.000427
1,Arenaviridae,13,86.3640,100.0,7.890000e-20,0.000206
12,Flaviviridae,10,92.0000,247.5,4.005000e-73,0.000158


In [17]:
with pd.ExcelWriter('output_2.xlsx') as writer:
    taxons_merged.to_excel(writer, sheet_name='taxons', index=None)
    virus_stats.to_excel(writer, sheet_name='stats', index=None)